In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# Going to Velcro search page in Usslide
options = webdriver.ChromeOptions()
options.add_argument("--headless")
browser = webdriver.Chrome(executable_path=r"chromedriver.exe",chrome_options=options)
#url = 'https://www.usslide.com/'
url = 'http://usslide.com/?s=velcro'
browser.get(url)

In [ ]:
data = []

hasNext = True
index = 0

while hasNext:

    # Get all HTML in document and assign it to BeautifulSoup variable
    innerHTML = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(innerHTML,"lxml")
    
    # Finds all of the individual products
    ul = soup.find('ul', 'search-results')
    items = ul.find_all('li')

    for link in range(len(items)):
        # Get all HTML in document and assign it to BeautifulSoup variable
        innerHTML = browser.execute_script('return document.body.innerHTML')
        soup = BeautifulSoup(innerHTML,"lxml")
        
        # Finds all of the individual products
        ul = soup.find('ul', 'search-results')
        items = ul.find_all('li')
        
        #Go to the product page
        item_urls = ul.find_all('a')        
        item_url = item_urls[index]
        browser.get(item_url['href'])        

        #Get the html
        innerHTML_product = browser.execute_script('return document.body.innerHTML')
        soup_product = BeautifulSoup(innerHTML_product,"lxml")        

        #Add to dictionary
        item = soup_product.find('div', 'entry-content')
        item_img = item.find('img')
        if item_img != None:
            item_img = item_img['src']
        else:
            item_img = 'No Image Found'
        item_name = item_url.get_text()        
        data += [{'product-image': item_img,
                  'product-name' : item_name.replace('\n', '').replace('\t','')}]        

        index += 1
        
        #Go back to the original page
        browser.back()

    # Get all HTML in document and assign it to BeautifulSoup variable
    innerHTML = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(innerHTML,"lxml")
    
    # If there is a next page go to the next page
    try:        
        nextLink = soup.find("a", 'next')        
        browser.get(nextLink['href'])
        index = 0
    except:
        hasNext = False